In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import copy
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from opacus import PrivacyEngine
from imblearn.over_sampling import SMOTE

# Load the KDD99 dataset (Assuming you have the dataset file 'kddcup.data_10_percent_corrected')
df = pd.read_csv('kddcup.data_10_percent_corrected', header=None)

# Define the column names based on KDD99 dataset features
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label'
]
df.columns = columns

# Step 1: Encode categorical features
for col in ['protocol_type', 'service', 'flag']:
    df[col] = LabelEncoder().fit_transform(df[col])

for col in ['protocol_type', 'service', 'flag']:
    df[col] = LabelEncoder().fit_transform(df[col])

# Step 2: Save original attack type labels
df['attack_type'] = df['label']  # Save string labels for later reporting

# Step 3: Encode the attack labels (multi-class classification)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label']) 


# Step 3: Standardize numerical features
scaler = StandardScaler()
df[df.columns[:-1]] = scaler.fit_transform(df[df.columns[:-1]])

# Step 4: Split the dataset into features and labels
X = df.drop(['label', 'attack_type'], axis=1).values
y = df['label'].values
attack_names = label_encoder.classes_

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create TensorDataset and DataLoader for IID setup (optional, use for evaluation)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Step 6: Simulating Non-IID Data Split for Clients
def split_noniid_data(X, y, num_clients):
    """
    Split data in a non-IID fashion among clients.
    Each client gets data with biased label distributions.
    """
    non_iid_data = []
    unique_labels = np.unique(y)
    
    # Split data by labels
    label_indices = {label: np.where(y == label)[0] for label in unique_labels}
    
    for client_id in range(num_clients):
        client_data_indices = []
        for label in unique_labels:
            # Each client gets a portion of data for each label (biased distribution)
            num_samples = int(len(label_indices[label]) / num_clients)
            selected_indices = np.random.choice(label_indices[label], num_samples, replace=False)
            client_data_indices.extend(selected_indices)
            
            # Remove selected indices to avoid overlap between clients
            label_indices[label] = np.setdiff1d(label_indices[label], selected_indices)
        
        # Add the client's data to the list
        client_data_X = X[client_data_indices]
        client_data_y = y[client_data_indices]
        non_iid_data.append((client_data_X, client_data_y))
    
    return non_iid_data

num_clients = 12
client_data_splits = split_noniid_data(X_train, y_train, num_clients)

# Example of how to convert each client's data to PyTorch tensors
client_datasets = []
for client_data_X, client_data_y in client_data_splits:
    client_X_tensor = torch.tensor(client_data_X, dtype=torch.float32)
    client_y_tensor = torch.tensor(client_data_y, dtype=torch.long)
    client_datasets.append(TensorDataset(client_X_tensor, client_y_tensor))


# Print client data sizes for verification
for i, dataset in enumerate(client_datasets):
    print(f"Client {i+1} data size: {len(dataset)} samples")


Client 1 data size: 32923 samples
Client 2 data size: 30181 samples
Client 3 data size: 27665 samples
Client 4 data size: 25362 samples
Client 5 data size: 23246 samples
Client 6 data size: 21310 samples
Client 7 data size: 19532 samples
Client 8 data size: 17905 samples
Client 9 data size: 16414 samples
Client 10 data size: 15043 samples
Client 11 data size: 13791 samples
Client 12 data size: 12641 samples


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import copy
from torch.utils.data import DataLoader
import torch.nn.functional as F
from opacus import PrivacyEngine  # Differential Privacy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, input_channels=1, output_size=2, input_length=41):  # Corrected __init__
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        
        # Calculate the flattened size after convolution
        self.fc_input_dim = 32 * input_length  # Adjust if pooling is added
        
        self.fc1 = nn.Linear(self.fc_input_dim, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        if x.ndim == 2:  # If shape is [batch_size, features]
            x = x.unsqueeze(1)  # Add a channel dimension: [batch_size, 1, features]
        elif x.ndim == 4:  # If shape is [batch_size, 1, 1, features]
            x = x.squeeze(2)  # Remove the extra dimension: [batch_size, 1, features]

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [7]:
def adversarial_attack(model, data, target, epsilon=0.1): #999
    data.requires_grad = True
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)
    model.zero_grad()
    loss.backward()
    perturbed_data = data + epsilon * data.grad.sign()
    return perturbed_data.detach()

In [13]:
class Client:
    def __init__(self, client_id, model, dataset, lr=0.001, mu=0.1, epsilon=0.2, delta=1e-5):
        self.client_id = client_id
        self.model = copy.deepcopy(model)
        self.dataset = dataset
        self.dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = nn.CrossEntropyLoss()
        self.mu = mu
        self.epsilon = epsilon
        self.delta = delta
        self.privacy_engine = PrivacyEngine()
        self.model, self.optimizer, self.dataloader = self.privacy_engine.make_private(
            module=self.model,
            optimizer=self.optimizer,
            data_loader=self.dataloader,
            noise_multiplier=0.3,
            max_grad_norm=1.5
        )
        self.grad_tracker = [torch.zeros_like(param) for param in self.model.parameters()]

    def train_local(self, global_model, epochs=1, adv_training=True, fkd=True):
        self.model.train()
        global_params = list(global_model.parameters())

        for epoch in range(epochs):
            for data, target in self.dataloader:
                data, target = data.to(torch.float32), target.to(torch.long)
                if adv_training:
                    data = adversarial_attack(self.model, data, target)
                self.optimizer.zero_grad()
                output = self.model(data)
                loss = self.criterion(output, target)

                # FedDyn regularization
                fed_dyn_reg = 0.0
                for param, g_param, z in zip(self.model.parameters(), global_params, self.grad_tracker):
                    fed_dyn_reg += torch.sum(param * (self.mu * (param - g_param.detach()) - z))
                loss += fed_dyn_reg

                if fkd:
                    with torch.no_grad():
                        global_output = global_model(data)
                    distill_loss = nn.KLDivLoss(reduction='batchmean')(
                        F.log_softmax(output, dim=1), F.softmax(global_output, dim=1)
                    )
                    loss += 0.4 * distill_loss

                loss.backward()
                self.optimizer.step()

        # Update the client’s historical gradient (FedDyn)
        with torch.no_grad():
            for i, (param, g_param) in enumerate(zip(self.model.parameters(), global_params)):
                self.grad_tracker[i] -= self.mu * (param.detach() - g_param.detach())

        return self.model.state_dict()


In [9]:
class Server:
    def __init__(self, model, num_clients):
        self.global_model = model
        self.num_clients = num_clients
        self.clients = []
        self.client_weights = []

    def register_client(self, client):
        self.clients.append(client)

    def weighted_aggregate(self, client_weights, client_data_sizes):
        total_data = sum(client_data_sizes)
        avg_weights = copy.deepcopy(client_weights[0])
        for key in avg_weights:
            avg_weights[key] = sum(
                client_weights[i][key] * (client_data_sizes[i] / total_data) for i in range(len(client_weights))
            )
        clean_weights = {key.replace("_module.", ""): val for key, val in avg_weights.items()}
        return clean_weights

    def federated_training(self, rounds=10, epochs=1, adv_training=True, fkd=True, dynamic_fed=True):
        for r in range(rounds):
            # Dynamic client selection (top 50% based on update quality or randomly)
            selected_clients = self.clients if not dynamic_fed else list(
                np.random.choice(self.clients, max(1, len(self.clients) // 2), replace=False)
            )

            client_weights = []
            client_data_sizes = []

            for client in selected_clients:
                local_state = client.train_local(self.global_model, epochs, adv_training, fkd)
                client_weights.append(local_state)
                client_data_sizes.append(len(client.dataset))  # used for weighted aggregation

            aggregated_weights = self.weighted_aggregate(client_weights, client_data_sizes)
            self.global_model.load_state_dict(aggregated_weights)
            print(f'[FedDyn] Round {r + 1} completed.')

    def evaluate_model(self, test_loader):
        self.global_model.eval()
        y_true, y_pred = [], []
        total_loss = 0.0
        criterion = nn.CrossEntropyLoss()

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(torch.float32), target.to(torch.long)
                output = self.global_model(data)
                total_loss += criterion(output, target).item()
                predictions = torch.argmax(output, dim=1)
                y_true.extend(target.numpy())
                y_pred.extend(predictions.numpy())
        with torch.no_grad():
            for i in range(0, len(X_test_tensor), 512):
                inputs = X_test_tensor[i:i+512]
                targets = y_test_tensor[i:i+512]
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.numpy())
                all_labels.extend(targets.numpy())
                print("Classification Report (Per Attack Type):")
                print(classification_report(all_labels, all_preds, target_names=attack_names))


            

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average='macro')
        rec = recall_score(y_true, y_pred, average='macro')
        f1 = f1_score(y_true, y_pred, average='macro')

        print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1 Score: {f1:.4f}")
        sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.show()

        return acc, prec, rec, f1


In [15]:
server_model = CNN(input_channels=1, output_size=2, input_length=X_train.shape[1])
server = Server(server_model, num_clients)

In [16]:
for i in range(num_clients):
    client = Client(i, server_model, client_datasets[i])
    server.register_client(client)

In [17]:
server.federated_training(rounds=35, epochs=5, adv_training=True, dynamic_fed=True)

Round 1 completed.
Round 2 completed.
Round 3 completed.
Round 4 completed.
Round 5 completed.
Round 6 completed.
Round 7 completed.
Round 8 completed.
Round 9 completed.
Round 10 completed.
Round 11 completed.
Round 12 completed.
Round 13 completed.
Round 14 completed.
Round 15 completed.
Round 16 completed.
Round 17 completed.
Round 18 completed.
Round 19 completed.
Round 20 completed.
Round 21 completed.
Round 22 completed.
Round 23 completed.
Round 24 completed.
Round 25 completed.
Round 26 completed.
Round 27 completed.
Round 28 completed.
Round 29 completed.
Round 30 completed.
Round 31 completed.
Round 32 completed.
Round 33 completed.
Round 34 completed.
Round 35 completed.


In [7]:
erver.evaluate_model(DataLoader(test_dataset, batch_size=32, shuffle=False),label_encoder)

Classification Report (5-Class KDD Categories):
              precision    recall  f1-score   support

         DoS       0.995      0.997      0.996     7450
      Normal       0.982      0.987      0.984     5500
       Probe       0.964      0.951      0.957      700
         R2L       0.842      0.761      0.799      200
         U2R       0.781      0.639      0.702       90

    accuracy                           0.9839    13940
   macro avg       0.913      0.867      0.888    13940
weighted avg       0.984      0.984      0.984    13940
